In [35]:
import cv2
import math
import numpy as np
import matplotlib.pylab as plt
from glob import glob

In [36]:
images_list = glob(r"KI/Bilder2/*.png")
#image = cv2.imread(images_list[30])
image = cv2.imread(r".\KI\Bilder2\PXL_20230414_105843090.jpg")
image = cv2.resize(image, (600,800))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [37]:
def auto_canny_edge_detection(image, sigma=0.33):
    md = np.median(image)
    lower_value = int(max(0, (1.0-sigma) * md))
    upper_value = int(min(255, (1.0+sigma) * md))
    return cv2.Canny(image, lower_value, upper_value)

auto_edge = auto_canny_edge_detection(blurred)

In [38]:
# Copy edges to the images that will display the results in BGR
dst = cv2.Canny(blurred, 50, 200, None, 3)
cdst = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
cdstP = np.copy(cdst)


# Probabilistic Line Transform
linesP = cv2.HoughLinesP(dst, 1, np.pi / 180, 20, 3, 2, 10)
# Draw the lines
if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv2.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)

In [39]:
# Apply Laplacian of Gaussian
blobs_log = cv2.Laplacian(cdstP, cv2.CV_64F)
blobs_log = np.uint8(np.absolute(blobs_log))
    
# Set up the detector with default parameters.
params = cv2.SimpleBlobDetector_Params()

# Change thresholds
params.minThreshold = 1
params.maxThreshold = 1000

# Filter by Area.
params.filterByArea = True
params.minArea = 800
params.maxArea = 5000

# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.01
params.maxCircularity = 1

# Filter by Convexity
params.filterByConvexity = True
params.minConvexity = 0.1
params.maxConvexity = 5

# Filter by Inertia
params.filterByInertia = True
params.minInertiaRatio = 0.01
params.maxInertiaRatio = 1

# Create a detector with the parameters
detector = cv2.SimpleBlobDetector_create(params)

# Detect blobs.
keypoints = detector.detect(blobs_log)

# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of the blob
im_with_keypoints = cv2.drawKeypoints(cdstP, keypoints, np.array([]), (0, 255, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

In [40]:
# Show keypoints
winname1="Keypoints"
cv2.namedWindow(winname1) 
cv2.moveWindow(winname1, 500,-2000)
cv2.imshow(winname1, im_with_keypoints)
cv2.waitKey()
cv2.destroyAllWindows()